In [1]:
import inspect
import types

class MultiMethod:
    """ Represents a multimethod (a function with multiple implementations). """
    
    def __init__(self, name):
        self._methods = {}  # Dictionary to store methods based on argument types
        self.__name__ = name  # Store method name

    def register(self, meth):
        """ Register a new method into the multimethod system. """
        sig = inspect.signature(meth)  # Get function signature

        types_list = []
        for name, param in sig.parameters.items():
            if name == 'self':
                continue
            if param.annotation is inspect.Parameter.empty:
                raise TypeError(f'Argument {name} must be annotated with a type')
            if not isinstance(param.annotation, type):
                raise TypeError(f'Argument {name} annotation must be a type')
            
            types_list.append(param.annotation)  # Store the type annotations
        
        self._methods[tuple(types_list)] = meth  # Map types to the function

    def __call__(self, *args):
        """ Call the method based on argument types. """
        types = tuple(type(arg) for arg in args[1:])  # Get argument types (excluding self)
        meth = self._methods.get(types, None)  # Find the matching method
        
        if meth:
            return meth(*args)
        else:
            raise TypeError(f'No matching method for types {types}')

    def __get__(self, instance, cls):
        """ Descriptor to support method binding. """
        if instance is not None:
            return types.MethodType(self, instance)  # Bind method to instance
        else:
            return self  # Return the unbound method

class MultiDict(dict):
    """ Special dictionary to build multimethods dynamically in metaclass. """

    def __setitem__(self, key, value):
        if key in self:
            current_value = self[key]
            if isinstance(current_value, MultiMethod):
                current_value.register(value)  # Register new method
            else:
                mvalue = MultiMethod(key)
                mvalue.register(current_value)
                mvalue.register(value)
                super().__setitem__(key, mvalue)
        else:
            super().__setitem__(key, value)  # Store method normally

class MultipleMeta(type):
    """ Metaclass enabling multiple dispatch. """

    def __new__(cls, clsname, bases, clsdict):
        return type.__new__(cls, clsname, bases, dict(clsdict))

    @classmethod
    def __prepare__(cls, clsname, bases):
        return MultiDict()  # Use custom dictionary


In [3]:
class Spam(metaclass=MultipleMeta):
    def bar(self, x: int, y: int):
        print('Bar 1:', x, y)

    def bar(self, s: str, n: int = 0):
        print('Bar 2:', s, n)

# Create object
s = Spam()

# Call methods with different argument types
s.bar(2, 3)         # Prints "Bar 1: 2 3"
s.bar('hello', 0)      # Prints "Bar 2: hello 0"
s.bar('hi', 5)      # Prints "Bar 2: hi 5"

# This will raise an error because no matching method is found
# s.bar(2, 'hello')   # TypeError: No matching method for types (<class 'int'>, <class 'str'>)


Bar 1: 2 3
Bar 2: hello 0
Bar 2: hi 5


In [4]:
import types

class multimethod:
    """ A decorator-based multiple dispatch implementation. """

    def __init__(self, func):
        self._methods = {}
        self.__name__ = func.__name__
        self._default = func  # Default method if no match found

    def match(self, *types):
        """ Register a method with specific argument types. """
        def register(func):
            ndefaults = len(func.__defaults__) if func.__defaults__ else 0
            for n in range(ndefaults + 1):
                self._methods[types[:len(types) - n]] = func
            return self
        return register

    def __call__(self, *args):
        """ Calls the correct method based on argument types. """
        types = tuple(type(arg) for arg in args[1:])  # Get types (excluding self)
        meth = self._methods.get(types, None)
        if meth:
            return meth(*args)
        else:
            return self._default(*args)  # Call default method if no match

    def __get__(self, instance, cls):
        if instance is not None:
            return types.MethodType(self, instance)
        else:
            return self

# Example usage
class Spam:
    @multimethod
    def bar(self, *args):
        """ Default method if no match found. """
        raise TypeError('No matching method for bar')

    @bar.match(int, int)
    def bar(self, x, y):
        print('Bar 1:', x, y)

    @bar.match(str, int)
    def bar(self, s, n=0):
        print('Bar 2:', s, n)

# Testing the decorator version
s = Spam()
s.bar(2, 3)         # Prints "Bar 1: 2 3"
s.bar('hello')      # Prints "Bar 2: hello 0"
s.bar('hi', 5)      # Prints "Bar 2: hi 5"
# s.bar(2, 'hello')  # Raises TypeError


Bar 1: 2 3
Bar 2: hello 0
Bar 2: hi 5
